gets name, focuscode and frequency range of all observatories of a specific day

In [2]:
from astropy.io import fits
import urllib
from bs4 import BeautifulSoup
import datetime

frq_limit_low = 50
frq_limit_high = 500

def listFilesDay(url):
    page = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(page, 'html.parser')
    return [node.get('href') for node in soup.find_all('a') if node.get('href').endswith('.fit.gz')]

# mydate = "/2022/01/26"
mydate = "{:%Y/%m/%d}".format(datetime.date.today())
url = 'http://soleil.i4ds.ch/solarradio/data/2002-20yy_Callisto/'
files = listFilesDay(url+mydate)

stations = []
for i in files:
    parts = i.rsplit("_")
    stations.append([parts[0], parts[3][:2]])
stations_clean = []
for i in stations:
    if i not in stations_clean:
        stations_clean.append(i)

def listFD(url, station):
    page = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(page, 'html.parser')

    return [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').startswith(station[0]) and node.get('href').endswith(station[1] + '.fit.gz')]

for i in stations_clean:
    for a, b in enumerate(listFD(url+mydate, i)):

        with fits.open(b) as fds:
            """
            try except -> print(fds[0].header["CDELT2"])
                          print(fds[0].header["CRVAL2"] - fds[0].header["CDELT2"] * fds[0].header["CRPIX2"])
            if fds[1] exists
            """
            lat = fds[0].header['OBS_LAT']
            lac = fds[0].header['OBS_LAC']
            if lac =='S':
                lat = -lat
            lon = fds[0].header['OBS_LON']
            loc = fds[0].header['OBS_LOC']
            if loc=='W':
                lon = -lon
            frq_axis = fds[1].data['frequency'].flatten()
            frq = sorted([frq_axis[0], frq_axis[-1]])
            if frq[0]<frq_limit_low and frq[1]<frq_limit_high:
                print(i[0],i[1], lon, lat, frq)
            else:
                print("discard: ---------------- ", i[0],i[1], lon, lat, frq)
            break


discard: ----------------  ALASKA-ANCHORAGE 01 -149.960006713867 61.2000007629395 [250.0, 453.81298828125]
ALASKA-COHOE 62 -151.320007324219 60.3699989318848 [5.0, 88.93800354003906]
ALASKA-COHOE 63 -151.320007324219 60.3699989318848 [5.0, 88.93800354003906]
ALASKA-HAARP 62 -145.169998168945 62.4000015258789 [5.0, 88.93800354003906]
ALASKA-HAARP 63 -145.169998168945 62.4000015258789 [5.0, 88.93800354003906]
ALGERIA-CRAAG 59 3.27999997138977 36.4599990844727 [45.0, 164.93800354003906]
ALMATY 58 76.5800018310547 43.0299987792969 [45.0, 164.5]
discard: ----------------  ALMATY 59 76.5800018310547 43.0299987792969 [175.0, 868.06298828125]
AUSTRIA-Krumbach 10 9.55000019073486 47.2900009155273 [45.0, 463.75]
discard: ----------------  AUSTRIA-MICHELBACH 60 15.3999996185303 48.0 [180.0, 383.5]
AUSTRIA-OE3FLB 57 15.3549995422363 48.0099983215332 [20.0, 91.625]
discard: ----------------  AUSTRIA-OE3FLB 55 15.3549995422363 48.0099983215332 [510.0, 784.625]
AUSTRIA-UNIGRAZ 01 15.5 47.099998474121

TODO: what frequency ranges are allowed to correlate 

TODO: rewrite observatories -> focus code part of class

TODO: write/load observatories to/from file